In [125]:
import random
import numpy as np

mental_states = ['happy', 'sad', 'stressed', 'relaxed']
num_users = 1000

user_mental_states = [random.choice(mental_states) for _ in range(num_users)]

In [113]:
preferences_database = {}

for mental_state in mental_states:
    preferences_database[mental_state] = {
        'books': [f'Book {i}' for i in range(1, 11)],  # Simulating 10 book preferences per mental state
        'films': [f'Film {i}' for i in range(1, 11)],  # Simulating 10 film preferences per mental state
        'music': [f'Music {i}' for i in range(1, 11)]  # Simulating 10 music preferences per mental state
    }


In [114]:
user_preferences = {}

for user_id, mental_state in enumerate(user_mental_states):
    user_preferences[user_id] = {
        'mental_state': mental_state,
        'preferences': {
            'books': random.sample(preferences_database[mental_state]['books'], 3),  # Selecting 3 random books
            'films': random.sample(preferences_database[mental_state]['films'], 3),  # Selecting 3 random films
            'music': random.sample(preferences_database[mental_state]['music'], 3)   # Selecting 3 random music
        }
    }


In [115]:
import csv
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
path = '/content/drive/MyDrive/FYP_RL_MODELS/RS USING RL BRANDIT/data.csv'

fieldnames = ['user_id', 'mental_state', 'books_1', 'books_2', 'books_3', 'films_1', 'films_2', 'films_3', 'music_1', 'music_2', 'music_3']

with open(path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write data for each user
    for user_id, data in user_preferences.items():
        row = {'user_id': user_id, 'mental_state': data['mental_state']}

        # Add preferences to row
        for category, items in data['preferences'].items():
            for i, item in enumerate(items):
                row[f"{category}_{i+1}"] = item

        # Write row to CSV file
        writer.writerow(row)

print(f"User preferences data has been written to {path}.")

User preferences data has been written to /content/drive/MyDrive/FYP_RL_MODELS/RS USING RL BRANDIT/data.csv.


In [117]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/FYP_RL_MODELS/RS USING RL BRANDIT/data.csv')
data.head()

,user_id,mental_state,books_1,books_2,books_3,films_1,films_2,films_3,music_1,music_2,music_3
0,0,happy,Book 8,Book 9,Book 6,Film 3,Film 5,Film 2,Music 1,Music 5,Music 4
1,1,happy,Book 5,Book 1,Book 3,Film 2,Film 6,Film 7,Music 7,Music 9,Music 1
2,2,relaxed,Book 2,Book 9,Book 1,Film 1,Film 7,Film 2,Music 5,Music 8,Music 2
3,3,stressed,Book 6,Book 10,Book 3,Film 8,Film 9,Film 1,Music 1,Music 10,Music 3
4,4,relaxed,Book 6,Book 5,Book 10,Film 10,Film 2,Film 1,Music 6,Music 9,Music 2


In [118]:
# books_mapping = {'Book 1': 1, 'Book 2': 2, 'Book 3': 3, 'Book 4': 4, 'Book 5': 5, 'Book 6': 6, 'Book 7': 7, 'Book 8': 8, 'Book 9': 9, 'Book 10': 10}
# films_mapping = {'Film 1': 1, 'Film 2': 2, 'Film 3': 3, 'Film 4': 4, 'Film 5': 5, 'Film 6': 6, 'Film 7': 7, 'Film 8': 8, 'Film 9': 9, 'Film 10': 10}
# music_mapping = {'Music 1': 1, 'Music 2': 2, 'Music 3': 3, 'Music 4': 4, 'Music 5': 5, 'Music 6': 6, 'Music 7': 7, 'Music 8': 8, 'Music 9': 9, 'Music 10': 10}
# mental_state_mapping = {'happy': 0, 'sad': 1, 'stressed': 2, 'relaxed': 3}

# # Map categorical values to numerical values
# data['books_1'] = data['books_1'].map(books_mapping)
# data['books_2'] = data['books_2'].map(books_mapping)
# data['books_3'] = data['books_3'].map(books_mapping)
# data['films_1'] = data['films_1'].map(films_mapping)
# data['films_2'] = data['films_2'].map(films_mapping)
# data['films_3'] = data['films_3'].map(films_mapping)
# data['music_1'] = data['music_1'].map(music_mapping)
# data['music_2'] = data['music_2'].map(music_mapping)
# data['music_3'] = data['music_3'].map(music_mapping)
# data['mental_state'] = data['mental_state'].map(mental_state_mapping)

# print(data.head())


In [119]:
# from sklearn.preprocessing import MinMaxScaler

# numerical_columns = ['mental_state', 'books_1', 'books_2', 'books_3', 'films_1', 'films_2', 'films_3', 'music_1', 'music_2', 'music_3']

# scaler = MinMaxScaler()

# data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# print(data.head())

In [120]:
def get_user_pref(df, user_id, mental_state):
  user_row = df[(df["user_id"] == user_id) & (df["mental_state"] == mental_state)]

  if not user_row.empty:
        user_row = user_row.iloc[0]
        books_preferences = [user_row["books_1"], user_row["books_2"], user_row["books_3"]]
        films_preferences = [user_row["films_1"], user_row["films_2"], user_row["films_3"]]
        music_preferences = [user_row["music_1"], user_row["music_2"], user_row["music_3"]]
  else:
        books_preferences = {}
        films_preferences = {}
        music_preferences = {}


  return books_preferences, films_preferences, music_preferences

In [121]:
class RecommendationEnvironment:
    def __init__(self, user_id, mental_state, df):
        self.user_id = user_id
        self.mental_state = mental_state
        self.books, self.films, self.music = get_user_pref(df, user_id, mental_state)
        self.current_state = self.get_state()

    def get_state(self):
        return {
            "user_id": self.user_id,
            "mental_state": self.mental_state,
            "books": self.books,
            "films": self.films,
            "music": self.music
        }



In [124]:
env = RecommendationEnvironment(0, 'happy', data)
print(env.get_state())

{'user_id': 0, 'mental_state': 'happy', 'books': ['Book 8', 'Book 9', 'Book 6'], 'films': ['Film 3', 'Film 5', 'Film 2'], 'music': ['Music 1', 'Music 5', 'Music 4']}


In [123]:
class RecommendationAgent:
    def __init__(self, recommendation_environment, initial_ratios=None, epsilon=0.1, alpha=0.1):
        self.env = recommendation_environment
        self.epsilon = epsilon
        self.alpha = alpha
        self.ratios = initial_ratios if initial_ratios else {"books": 1/3, "films": 1/3, "music": 1/3}
        self.Q_values = {"books": np.zeros(len(self.env.books)),
                         "films": np.zeros(len(self.env.films)),
                         "music": np.zeros(len(self.env.music))}
        self.action_counts = {"books": np.zeros(len(self.env.books)),
                              "films": np.zeros(len(self.env.films)),
                              "music": np.zeros(len(self.env.music))}


    def epsilon_greedy_action(self, category):
        if np.random.rand() < self.epsilon:
            return np.random.choice(len(self.env.get_state()[category]))
        else:
            return np.argmax(self.Q_values[category])


    def recommend(self):
      num_books = int(len(self.env.books) * self.ratios["books"])
      num_films = int(len(self.env.films) * self.ratios["films"])
      num_music = int(len(self.env.music) * self.ratios["music"])

      recommended_books_idx = self.epsilon_greedy_action("books")
      recommended_films_idx = self.epsilon_greedy_action("films")
      recommended_music_idx = self.epsilon_greedy_action("music")
      recommended_books = self.env.books[recommended_books_idx]
      recommended_films = self.env.films[recommended_films_idx]
      recommended_music = self.env.music[recommended_music_idx]

      return recommended_books, recommended_films, recommended_music


    def adjust_ratios(self, feedback):
      if feedback == "thumbs_up":
            for category in self.ratios:
                self.ratios[category] *= 1.1

      elif feedback == "skip":
            for category in self.ratios:
                self.ratios[category] *= 0.9

      elif feedback == "select":
        pass


    def get_reward(self, feedback):
      if feedback == "thumbs_up":
            return 1.0  # High reward for thumbs up
      elif feedback == "skip":
            return -0.5  # Low reward for skip
      else:
           return 0.0  # Medium reward for select or no feedback

